In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score

In [ ]:
raw_data = pd.read_csv("fifa21_male2.csv")
raw_data.head()

In [ ]:
raw_data.info()

In [ ]:
pd.options.display.max_columns = None
raw_data.head()

In [ ]:
raw_data.info(verbose=True)

In [ ]:
#standardising column names
raw_data.columns = [name.lower().replace(" ", "_") for name in raw_data.columns]
raw_data.head()

In [ ]:
# converting measurements to numerical variables 
# we have to change the columns 14, 15, 20-22 "height", "weight", "value", "wage", "clause"

# first, we'll remove the feet and inch symbols and multiply by 2.54. NB: height is now expressed in centimetres

raw_data['height'] = (raw_data['height']
 .str.extract(r'(\d+)\'\s*(\d+)"')
 .astype(float)
 .mul([12*2.54, 2.54])
 .sum(axis=1)
 )

raw_data.head()


#source: https://stackoverflow.com/questions/73007247/converting-inches-to-cm-on-series

In [ ]:
# change pounds "lbs""

raw_data['weight'] = (raw_data['weight']
.str.replace('lbs$|ml$', '')
)

raw_data.head()


In [ ]:
# change pounds to kilos

raw_data['weight'] = (raw_data['weight'].astype(float)) # convert to a number

raw_data['weight'] = raw_data['weight'] * 0.45359237 #  mutliply by .45

#check that the updated column is there
raw_data.head()

In [ ]:
# converting money amounts to numbers
raw_data['value'].value_counts() # see what we have
# we have a mix of millions and thousands 

In [ ]:
# remove euro symbol
#raw_data['value'] = raw_data['value'].apply(lambda x: string.replace("€",""))
raw_data['value'] = raw_data['value'].replace('\u20AC','',regex=True)
raw_data.head()

In [ ]:
# convert value from symbols to numbers
# create a function to remove K and M strings, convert string to numbers and mutliply by 1M or 1K
def str_to_num(x):
    if 'K' in x:
        return float(x.replace('K','')) * 1000
    elif 'M' in x:
        return float(x.replace('M','')) * 1000000
    else:
        return float(x)

# apply the function to the column
raw_data['value'] = raw_data['value'].apply(str_to_num)
raw_data.head()

In [ ]:
# convert wages and release clause from symbols to numbers
# create a function to remove K and M strings, convert string to numbers and mutliply by 1M or 1K

raw_data['wage'] = raw_data['wage'].replace('\u20AC','',regex=True)
raw_data['release_clause'] = raw_data['release_clause'].replace('\u20AC','',regex=True)


def str_to_num(x):
    if 'K' in x:
        return float(x.replace('K','')) * 1000
    elif 'M' in x:
        return float(x.replace('M','')) * 1000000
    else:
        return float(x)

# apply the function to the column
raw_data['wage'] = raw_data['wage'].apply(str_to_num)
raw_data['release_clause'] = raw_data['release_clause'].apply(str_to_num)

raw_data.head()

In [ ]:
#Jesus: here I am splitting the contract column into two columns "contract_start", "contract_end"
raw_data[['contract_start', 'contract_end']] = raw_data['contract'].str.split(' ~ ', expand=True)

raw_data['contract_end'].fillna(value=np.nan, inplace=True)

raw_data.drop(['contract', 'team_&_contract'], axis=1, inplace=True)

print(raw_data.loc[:, ['contract_start', 'contract_end']])

In [ ]:
#Jesus: here I am looking for values that are not years in "contract_start", "contract_end" to understand how many invalid values we have
pattern = r'^\d{4}$'

raw_data['contract_end'] = raw_data['contract_end'].astype(str)

start_count = raw_data[~raw_data['contract_start'].str.contains(pattern)].shape[0]
end_count = raw_data[~raw_data['contract_end'].str.contains(pattern)].shape[0]

print(f"Rows with non-year format in contract_start: {start_count}")
print(f"Rows with non-year format in contract_end: {end_count}")

In [ ]:
#Jesus: here I am dropping invalid values in "contract_start", "contract_end"
pattern = r'^\d{4}$'

raw_data['contract_end'] = raw_data['contract_end'].astype(str)

mask = ~(raw_data['contract_start'].str.contains(pattern)) | ~(raw_data['contract_end'].str.contains(pattern))

raw_data = raw_data.drop(index=raw_data[mask].index)

print(raw_data.loc[:, ['contract_start', 'contract_end']])

In [ ]:
#Jesus: here I am converting 'contract_start' and 'contract_end' to int
raw_data['contract_start'] = pd.to_numeric(raw_data['contract_start'])
raw_data['contract_end'] = pd.to_numeric(raw_data['contract_end'])

print(raw_data.loc[:, ['contract_start', 'contract_end']])

### Dealing with Empty Values (Ricardo)

In [ ]:
#creating the list of columns for numericals and non numericals
num_cols = raw_data.select_dtypes(include=['float64', 'int64']).columns
non_num_cols = raw_data.select_dtypes(exclude=['float64', 'int64']).columns

In [ ]:
#filling empty values with the mean for the numeric ones
raw_data[num_cols] = raw_data[num_cols].fillna(raw_data[num_cols].mean())

In [ ]:
non_num_cols

In [ ]:
for col in non_num_cols:
    if col in raw_data.columns:
        mode = raw_data[col].mode()
        if not mode.empty:
            raw_data[col].fillna(mode[0], inplace=True)
        else:
            print(f"No mode found for column {col}")
    else:
        print(f"Column {col} not found in raw_data")

### Checking for Multicolianerity and Removing Columns (Ricardo)

In [ ]:
#splitting numerical and categorical
X_num = raw_data.select_dtypes(include = np.number)
X_cat = raw_data.select_dtypes(include = np.object)
X_num

In [ ]:
# Creating the correlation matrix
correlation_matrix = X_num.corr().apply(lambda x: round(x,2))

In [ ]:
#visually presenting correlation matrix
sns.heatmap(correlation_matrix, cmap='coolwarm')
plt.show()

In [ ]:
#deleting columns thay have correlation higher than 0.7 with each other
high_corr_cols = set()

for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) >= 0.70:
            colname = correlation_matrix.columns[i]
            high_corr_cols.add(colname)

In [ ]:
#deleting columns thay have correlation higher than 0.7 with each other
raw_data = raw_data.drop(columns=high_corr_cols)

In [ ]:
#deleting other useless columns
raw_data =raw_data.drop(columns = ["Player Photo" , "Club Logo","Flag Photo", "Gender","ID"])
raw_data